In [1]:
from sklearn.ensemble import AdaBoostRegressor
from keras.models import Sequential, Model
from keras import layers, Input
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import KFold,StratifiedKFold


Using TensorFlow backend.


In [2]:
# 读取
#data = pd.read_excel("相邻位置仅留PM和T-1.xlsx")
data = pd.read_excel("自身与相邻站点PM_AOD_T-1_全样本.xlsx")
#data = pd.read_excel("测试用数据.xlsx")

In [3]:
# 设置变量
#data = data[data["AOD值"] > 0]
data = data[data["日均PM2.5"] > 0]

independent = ["AOD值", 'cloudCover', 'dewPoint', 'humidity', 'precipAccumulation', 'precipIntensity', 'pressure',
               'temperature', 'uvIndex', 'visibility', 'windSpeed', 'windBearing']
T_1 = ["AOD值-t-1", 'cloudCover-t-1', 'dewPoint-t-1', 'humidity-t-1', 'precipAccumulation-t-1', 'precipIntensity-t-1',
       'pressure-t-1', 'temperature-t-1', 'uvIndex-t-1', 'visibility-t-1', 'windSpeed-t-1', 'windBearing-t-1']
PM_list = ["A1-日均PM2.5-MEAN-t-1", "A2-日均PM2.5-MEAN-t-1", "A3-日均PM2.5-MEAN-t-1", "A4-日均PM2.5-MEAN-t-1",
           "A5-日均PM2.5-MEAN-t-1", "A6-日均PM2.5-MEAN-t-1", "A7-日均PM2.5-MEAN-t-1", "A8-日均PM2.5-MEAN-t-1",
           "B1-日均PM2.5-MEAN-t-1", "B2-日均PM2.5-MEAN-t-1", "B3-日均PM2.5-MEAN-t-1", "B4-日均PM2.5-MEAN-t-1",
           "B5-日均PM2.5-MEAN-t-1", "B6-日均PM2.5-MEAN-t-1", "B7-日均PM2.5-MEAN-t-1", "B8-日均PM2.5-MEAN-t-1",
           "日均PM2.5-t-1"]
dependent = ["日均PM2.5"]
# independent = list(set(independent) | set(T_1))  # 合集


In [ ]:
# 输入1和2的变量数,维度
inputA = Input(shape=(12,))
inputB = Input(shape=(17,))
inputC = Input(shape=(12,))
# 输入1
x = layers.Dense(8, activation="relu")(inputA)
x = layers.Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
# 输入2
y = layers.Dense(64, activation="relu")(inputB)
y = layers.Dense(32, activation="relu")(y)
y = layers.Dense(4, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)
# 输入3
x3 = layers.Dense(8, activation="relu")(inputC)
x3 = layers.Dense(4, activation="relu")(x3)
x3 = Model(inputs=inputC, outputs=x3)
# 合并多输入
combined = layers.concatenate([x.output, y.output, x3.input])
# 输出层
z = layers.Dense(4, activation="relu")(combined)
z = layers.Dense(1, activation="linear")(z)
# 建立模型
model = Model(inputs=[x.input, y.input, x3.input], outputs=z)
# 模型编译
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

W0629 19:15:26.857666  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0629 19:15:26.874621  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0629 19:15:26.880605  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0629 19:15:27.279016  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
print('#######打乱数据#######')
# 打乱
data = shuffle(data)  # , random_state=0
print("样本量:", len(data))

print('#######进行K折分组#######')
# k折分组,训练和测试 9:1
kf = KFold(n_splits=10)  # 参数shuffle=True

#######打乱数据#######
样本量: 128107
#######进行K折分组#######


In [ ]:
error_AME = []
error_MSE = []
error_RE = []
for train, test in kf.split(data):

    # 划分
    x1_train = data.iloc[train][independent]
    x1_test = data.iloc[test][independent]
    x2_train = data.iloc[train][PM_list]
    x2_test = data.iloc[test][PM_list]
    x3_train = data.iloc[train][T_1]
    x3_test = data.iloc[test][T_1]
    y_train = data.iloc[train][dependent]
    y_test = data.iloc[test][dependent]

    # np 格式

    x1_test_np = np.array(x1_test)
    x1_train_np = np.array(x1_train)
    x2_test_np = np.array(x2_test)
    x2_train_np = np.array(x2_train)
    x3_test_np = np.array(x3_test)
    x3_train_np = np.array(x3_train)
    y_test_np = np.array(y_test)
    y_train_np = np.array(y_train)

    # 模型计算
    '''
    ensemble = AdaBoostRegressor(base_estimator=model, learning_rate=0.001,
                                 loss='linear')
    ensemble.fit(X=[x1_train_np, x2_train_np], y=y_train_np)
    res = ensemble.predict([x1_test_np, x2_test_np])

    '''
    model.fit([x1_train_np, x2_train_np, x3_train_np], y_train_np, epochs=1000, batch_size=2000)
    res = model.predict([x1_test_np, x2_test_np, x3_test_np])

    res = pd.DataFrame(res)
    y_test = pd.DataFrame(y_test)
    # 相同索引方便合并
    res.index = y_test.index
    data_pred = pd.concat([res, y_test], axis=1)
    data_pred.columns = ["pre", "true"]
    # 计算误差
    e_AME = abs(data_pred["pre"] - data_pred["true"]).mean()
    # print("AME误差:", e)
    e_MSE = ((data_pred["pre"] - data_pred["true"]) ** 2).mean()
    error_AME.append(e_AME)
    error_MSE.append(e_MSE)
    # 相对误差百分比
    r_e = (abs(data_pred["pre"] - data_pred["true"])/abs(data_pred["true"])).mean()
    error_RE.append(r_e)
print("交叉验证后的平均AME误差值:", np.average(error_AME), "预测结果的标准差", np.std(error_AME))
print("交叉验证后的平均MSE误差值:", np.average(error_MSE), "预测结果的标准差", np.std(error_MSE))
print("相对误差:", np.average(error_RE),  "预测结果的标准差", np.std(error_RE))

W0629 19:15:28.304306  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0629 19:15:29.326167  6916 deprecation_wrapper.py:119] From D:\DataScience\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/1000
115296/115296 [==============================] - 1s 7us/step - loss: 78398.6058 - acc: 0.0017
Epoch 2/1000
115296/115296 [==============================] - 0s 3us/step - loss: 23465.3473 - acc: 0.0048
Epoch 3/1000
115296/115296 [==============================] - 0s 3us/step - loss: 10790.0238 - acc: 0.0065
Epoch 4/1000
115296/115296 [==============================] - 0s 3us/step - loss: 6212.9049 - acc: 0.0068
Epoch 5/1000
115296/115296 [==============================] - 0s 3us/step - loss: 4310.4563 - acc: 0.0069
Epoch 6/1000
115296/115296 [==============================] - 0s 3us/step - loss: 3506.5717 - acc: 0.0075
Epoch 7/1000
115296/115296 [==============================] - 0s 3us/step - loss: 3040.6274 - acc: 0.0087
Epoch 8/1000
115296/115296 [==============================] - 0s 3us/step - loss: 2765.2850 - acc: 0.0094
Epoch 9/1000
115296/115296 [==============================] - 0s 3us/step - loss: 2578.4035 - acc: 0.0099
Epoch 10/1000
115296/115296 [==============

115296/115296 [==============================] - 0s 3us/step - loss: 1050.9458 - acc: 0.0176
Epoch 78/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1044.6249 - acc: 0.0175
Epoch 79/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1044.4955 - acc: 0.0177
Epoch 80/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1041.9058 - acc: 0.0174
Epoch 81/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1036.4963 - acc: 0.0172
Epoch 82/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1034.5230 - acc: 0.0174
Epoch 83/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1035.1229 - acc: 0.0175
Epoch 84/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1030.8589 - acc: 0.0178
Epoch 85/1000
115296/115296 [==============================] - 0s 3us/step - loss: 1025.6616 - acc: 0.0181
Epoch 86/1000
115296/115296 [======================

115296/115296 [==============================] - 0s 3us/step - loss: 906.9695 - acc: 0.0190
Epoch 154/1000
115296/115296 [==============================] - 0s 3us/step - loss: 902.5867 - acc: 0.0188
Epoch 155/1000
115296/115296 [==============================] - 0s 3us/step - loss: 906.6984 - acc: 0.0195
Epoch 156/1000
115296/115296 [==============================] - 0s 3us/step - loss: 901.8896 - acc: 0.0189
Epoch 157/1000
115296/115296 [==============================] - 0s 3us/step - loss: 898.8946 - acc: 0.0192
Epoch 158/1000
115296/115296 [==============================] - 0s 3us/step - loss: 898.3386 - acc: 0.0190
Epoch 159/1000
115296/115296 [==============================] - 0s 3us/step - loss: 897.5168 - acc: 0.0191
Epoch 160/1000
115296/115296 [==============================] - 0s 3us/step - loss: 897.7258 - acc: 0.0194
Epoch 161/1000
115296/115296 [==============================] - 0s 3us/step - loss: 898.0956 - acc: 0.0191
Epoch 162/1000
115296/115296 [======================

115296/115296 [==============================] - 0s 3us/step - loss: 844.8195 - acc: 0.0197
Epoch 230/1000
115296/115296 [==============================] - 0s 3us/step - loss: 854.6201 - acc: 0.0194
Epoch 231/1000
115296/115296 [==============================] - 0s 3us/step - loss: 844.8288 - acc: 0.0201
Epoch 232/1000
115296/115296 [==============================] - 0s 3us/step - loss: 847.3882 - acc: 0.0203
Epoch 233/1000
115296/115296 [==============================] - 0s 3us/step - loss: 843.9854 - acc: 0.0199
Epoch 234/1000
115296/115296 [==============================] - 0s 3us/step - loss: 842.2475 - acc: 0.0197
Epoch 235/1000
115296/115296 [==============================] - 0s 3us/step - loss: 843.7071 - acc: 0.0207
Epoch 236/1000
115296/115296 [==============================] - 0s 3us/step - loss: 844.5291 - acc: 0.0205
Epoch 237/1000
115296/115296 [==============================] - 0s 3us/step - loss: 845.3433 - acc: 0.0198
Epoch 238/1000
115296/115296 [======================

...

115296/115296 [==============================] - 0s 3us/step - loss: 618.0837 - acc: 0.0240
Epoch 866/1000
115296/115296 [==============================] - 0s 3us/step - loss: 619.3850 - acc: 0.0242
Epoch 867/1000
115296/115296 [==============================] - 0s 3us/step - loss: 619.3995 - acc: 0.0240
Epoch 868/1000
115296/115296 [==============================] - 0s 3us/step - loss: 622.4915 - acc: 0.0238
Epoch 869/1000
115296/115296 [==============================] - 0s 3us/step - loss: 618.6191 - acc: 0.0246
Epoch 870/1000
115296/115296 [==============================] - 0s 2us/step - loss: 618.7491 - acc: 0.0243
Epoch 871/1000
115296/115296 [==============================] - 0s 2us/step - loss: 621.8452 - acc: 0.0239
Epoch 872/1000
115296/115296 [==============================] - 0s 2us/step - loss: 618.4428 - acc: 0.0241
Epoch 873/1000
115296/115296 [==============================] - 0s 2us/step - loss: 618.2646 - acc: 0.0237
Epoch 874/1000
115296/115296 [======================

115296/115296 [==============================] - 0s 2us/step - loss: 616.4145 - acc: 0.0232
Epoch 942/1000
115296/115296 [==============================] - 0s 2us/step - loss: 620.7254 - acc: 0.0232
Epoch 943/1000
115296/115296 [==============================] - 0s 2us/step - loss: 618.7459 - acc: 0.0246
Epoch 944/1000
115296/115296 [==============================] - 0s 2us/step - loss: 624.4323 - acc: 0.0245
Epoch 945/1000
115296/115296 [==============================] - 0s 2us/step - loss: 619.9356 - acc: 0.0235
Epoch 946/1000
115296/115296 [==============================] - 0s 2us/step - loss: 618.3626 - acc: 0.0236
Epoch 947/1000
115296/115296 [==============================] - 0s 2us/step - loss: 620.2508 - acc: 0.0234
Epoch 948/1000
115296/115296 [==============================] - 0s 2us/step - loss: 620.0704 - acc: 0.0239
Epoch 949/1000
115296/115296 [==============================] - 0s 2us/step - loss: 616.3586 - acc: 0.0238
Epoch 950/1000
115296/115296 [======================

115296/115296 [==============================] - 0s 2us/step - loss: 615.6249 - acc: 0.0234
Epoch 18/1000
115296/115296 [==============================] - 0s 2us/step - loss: 615.4121 - acc: 0.0240
Epoch 19/1000
115296/115296 [==============================] - 0s 2us/step - loss: 613.3625 - acc: 0.0249
Epoch 20/1000
115296/115296 [==============================] - 0s 2us/step - loss: 615.5544 - acc: 0.0237
Epoch 21/1000
115296/115296 [==============================] - 0s 2us/step - loss: 615.4711 - acc: 0.0236
Epoch 22/1000
115296/115296 [==============================] - 0s 2us/step - loss: 617.2386 - acc: 0.0235
Epoch 23/1000
115296/115296 [==============================] - 0s 2us/step - loss: 614.2028 - acc: 0.0244
Epoch 24/1000
115296/115296 [==============================] - 0s 2us/step - loss: 616.1500 - acc: 0.0241
Epoch 25/1000
115296/115296 [==============================] - 0s 2us/step - loss: 616.1287 - acc: 0.0239
Epoch 26/1000
115296/115296 [==============================]

115296/115296 [==============================] - 0s 2us/step - loss: 612.9306 - acc: 0.0244
Epoch 95/1000
115296/115296 [==============================] - 0s 2us/step - loss: 612.2309 - acc: 0.0237
Epoch 96/1000
115296/115296 [==============================] - 0s 2us/step - loss: 613.9040 - acc: 0.0236
Epoch 97/1000
115296/115296 [==============================] - 0s 2us/step - loss: 614.2824 - acc: 0.0240
Epoch 98/1000
115296/115296 [==============================] - 0s 2us/step - loss: 611.7179 - acc: 0.0237
Epoch 99/1000
115296/115296 [==============================] - 0s 2us/step - loss: 614.0940 - acc: 0.0241
Epoch 100/1000
115296/115296 [==============================] - 0s 2us/step - loss: 615.6069 - acc: 0.0241
Epoch 101/1000
115296/115296 [==============================] - 0s 2us/step - loss: 613.0949 - acc: 0.0236
Epoch 102/1000
115296/115296 [==============================] - 0s 2us/step - loss: 612.4838 - acc: 0.0243
Epoch 103/1000
115296/115296 [===========================

115296/115296 [==============================] - 0s 2us/step - loss: 612.2815 - acc: 0.0235
Epoch 171/1000
115296/115296 [==============================] - 0s 2us/step - loss: 612.5832 - acc: 0.0230
Epoch 172/1000
115296/115296 [==============================] - 0s 3us/step - loss: 610.2769 - acc: 0.0237
Epoch 173/1000
115296/115296 [==============================] - 0s 3us/step - loss: 611.5367 - acc: 0.0247
Epoch 174/1000
115296/115296 [==============================] - 0s 3us/step - loss: 616.4729 - acc: 0.0237
Epoch 175/1000
115296/115296 [==============================] - 0s 2us/step - loss: 612.6221 - acc: 0.0244
Epoch 176/1000
115296/115296 [==============================] - 0s 2us/step - loss: 609.9936 - acc: 0.0235
Epoch 177/1000
115296/115296 [==============================] - 0s 2us/step - loss: 612.2639 - acc: 0.0243
Epoch 178/1000
115296/115296 [==============================] - 0s 2us/step - loss: 614.1975 - acc: 0.0244
Epoch 179/1000
115296/115296 [======================